Part 1: Download a list of all European airports

Use the Amadeus Developers api: https://developers.amadeus.com/ 

Create a complete ETL process (from RAW to STD) and use GCloud Storage + MySQL database

In [ ]:
!pip install amadeus

In [2]:
# Amadeus Documentation

from amadeus import Client, Response

amadeus = Client(
    client_id='****************',
    client_secret='***********'
)

In [3]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [4]:
# Web Scraping of EU country codes to pass into API call

link = "https://www.destatis.de/Europa/EN/Country/Country-Codes.html"
headers = {"User-Agent": "Mozilla/5.0"}
req = Request(link, headers=headers)
raw_web = urlopen(req).read()

In [5]:
soup = BeautifulSoup(raw_web, 'html.parser')
table = soup.find('tbody')

In [6]:
rows = []

for tr in table.find_all('tr'):
    code = [td.text.strip() for td in tr.find_all('td')]
    rows.append(code)

In [7]:
codes = [i[1][:2] for i in rows]
codes

['AT',
 'BE',
 'BG',
 'HR',
 'CY',
 'CZ',
 'DK',
 'EE',
 'FI',
 'FR',
 'DE',
 'GR',
 'HU',
 'IE',
 'IT',
 'LV',
 'LT',
 'LU',
 'MT',
 'NL',
 'PL',
 'PT',
 'RO',
 'SK',
 'SI',
 'ES',
 'SE',
 'EU']

In [8]:
# Letters in alphabet to pass into "keyword" parameter of API call

import string

alphabet = list(string.ascii_uppercase)

In [12]:
# Save data received from API call to a list

airports=[]

for code in codes:

    for letter in alphabet:

        try:
            a = amadeus.reference_data.locations.get(
            keyword=letter,
            subType="AIRPORT",
            countryCode=code
            )
            airports.append(a)

        except:

            pass

In [22]:
airports

In [28]:
# Results in format of response objects -> use Amadeus documentation to find method of retrieving json body of the data

raw_data = []

for airport in airports:
    
    a = airport.result
    raw_data.append(a)

In [29]:
raw_data

[{'meta': {'count': 0,
   'links': {'self': 'https://test.api.amadeus.com/v1/reference-data/locations?subType=AIRPORT&keyword=A&countryCode=AT'}},
  'data': []},
 {'meta': {'count': 0,
   'links': {'self': 'https://test.api.amadeus.com/v1/reference-data/locations?subType=AIRPORT&keyword=B&countryCode=AT'}},
  'data': []},
 {'meta': {'count': 0,
   'links': {'self': 'https://test.api.amadeus.com/v1/reference-data/locations?subType=AIRPORT&keyword=C&countryCode=AT'}},
  'data': []},
 {'meta': {'count': 0,
   'links': {'self': 'https://test.api.amadeus.com/v1/reference-data/locations?subType=AIRPORT&keyword=E&countryCode=AT'}},
  'data': []},
 {'meta': {'count': 0,
   'links': {'self': 'https://test.api.amadeus.com/v1/reference-data/locations?subType=AIRPORT&keyword=F&countryCode=AT'}},
  'data': []},
 {'meta': {'count': 0,
   'links': {'self': 'https://test.api.amadeus.com/v1/reference-data/locations?subType=AIRPORT&keyword=H&countryCode=AT'}},
  'data': []},
 {'meta': {'count': 0,
   'l

In [33]:
len(raw_data)

624

In [34]:
# Some counts are = 0, meaning the API is limited to certain countries. Let us only include those

clean_data = []

for item in raw_data:
    
    if item['meta']['count'] > 0:
        clean_data.append(item)

In [36]:
len(clean_data)

78

In [37]:
clean_data

[{'meta': {'count': 21,
   'links': {'self': 'https://test.api.amadeus.com/v1/reference-data/locations?subType=AIRPORT&keyword=A&countryCode=FR',
    'next': 'https://test.api.amadeus.com/v1/reference-data/locations?subType=AIRPORT&keyword=A&countryCode=FR&page%5Boffset%5D=10',
    'last': 'https://test.api.amadeus.com/v1/reference-data/locations?subType=AIRPORT&keyword=A&countryCode=FR&page%5Boffset%5D=11'}},
  'data': [{'type': 'location',
    'subType': 'AIRPORT',
    'name': 'COTE D AZUR',
    'detailedName': 'NICE/FR:COTE D AZUR',
    'id': 'ANCE',
    'self': {'href': 'https://test.api.amadeus.com/v1/reference-data/locations/ANCE',
     'methods': ['GET']},
    'timeZoneOffset': '+01:00',
    'iataCode': 'NCE',
    'geoCode': {'latitude': 43.65834, 'longitude': 7.21584},
    'address': {'cityName': 'NICE',
     'cityCode': 'NCE',
     'countryName': 'FRANCE',
     'countryCode': 'FR',
     'regionCode': 'EUROP'},
    'analytics': {'travelers': {'score': 8}}},
   {'type': 'locatio

In [51]:
total_count = 0

for i in clean_data:
    count = i['meta']['count']
    total_count += count
    
print(total_count)

917


In [52]:
# Since it appears there are duplicates, likely from how we approached the API call in the double for loop
# let us make a dictionary which would eliminate these duplicates (in this case "airport id") since a dict cannot have duplicate keys

unique_airports = {} 

for country in clean_data:
        
    for airport in country['data']:
            
            unique_airports[airport['id']] = airport
      
            
unique_airports

{'ANCE': {'type': 'location',
  'subType': 'AIRPORT',
  'name': 'COTE D AZUR',
  'detailedName': 'NICE/FR:COTE D AZUR',
  'id': 'ANCE',
  'self': {'href': 'https://test.api.amadeus.com/v1/reference-data/locations/ANCE',
   'methods': ['GET']},
  'timeZoneOffset': '+01:00',
  'iataCode': 'NCE',
  'geoCode': {'latitude': 43.65834, 'longitude': 7.21584},
  'address': {'cityName': 'NICE',
   'cityCode': 'NCE',
   'countryName': 'FRANCE',
   'countryCode': 'FR',
   'regionCode': 'EUROP'},
  'analytics': {'travelers': {'score': 8}}},
 'ANTE': {'type': 'location',
  'subType': 'AIRPORT',
  'name': 'ATLANTIQUE',
  'detailedName': 'NANTES/FR:ATLANTIQUE',
  'id': 'ANTE',
  'self': {'href': 'https://test.api.amadeus.com/v1/reference-data/locations/ANTE',
   'methods': ['GET']},
  'timeZoneOffset': '+01:00',
  'iataCode': 'NTE',
  'geoCode': {'latitude': 47.15306, 'longitude': -1.61083},
  'address': {'cityName': 'NANTES',
   'cityCode': 'NTE',
   'countryName': 'FRANCE',
   'countryCode': 'FR',
 

In [53]:
len(unique_airports)

264

In [4]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="credentials.json"

In [5]:
from google.cloud import storage
import json

In [6]:
# Functions to create client and blob from GCS -> allows us to upload and download from our gcs bucket

def save_gcloud(_data, _name, _ruta):
    
    client = storage.Client()
    bucket = client.bucket('nov-2022')
    
    blob = bucket.blob(_ruta + _name)
    blob.upload_from_string(_data)
    
    print('Uploaded successfully')
    
def getfileslist(_carpeta):
    
    client = storage.Client()
    blobs = client.list_blobs('nov-2022', prefix = _carpeta)
    
    lista = []
    
    for blob in blobs:
        
        lista.append(blob.name)
        
    return lista

def getfile(_blob):
    
    client = storage.Client()
    bucket = client.bucket('imf-nov-2022')
    
    blob = bucket.blob(_blob)
    content = blob.download_as_string()
    
    return content

def txt_2_json(_content):
    
    return json.loads(_content)

In [60]:
ruta = 'airports/'

for i in unique_airports:
    
    airport = json.dumps(unique_airports[i])
    file = unique_airports[i]['name'] + '.json'
    
    save_gcloud(airport, file, ruta)

Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded successfully
Uploaded s

In [7]:
airports_list = []

for airport in getfileslist('airports/'):

    airports_list.append(txt_2_json(getfile(airport)))
    
airports_list

[{'type': 'location',
  'subType': 'AIRPORT',
  'name': 'A CORUNA AIRPORT',
  'detailedName': 'A CORUNA/ES:A CORUNA AIRPORT',
  'id': 'ALCG',
  'self': {'href': 'https://test.api.amadeus.com/v1/reference-data/locations/ALCG',
   'methods': ['GET']},
  'timeZoneOffset': '+01:00',
  'iataCode': 'LCG',
  'geoCode': {'latitude': 43.30195, 'longitude': -8.37722},
  'address': {'cityName': 'A CORUNA',
   'cityCode': 'LCG',
   'countryName': 'SPAIN',
   'countryCode': 'ES',
   'regionCode': 'EUROP'},
  'analytics': {'travelers': {'score': 1}}},
 {'type': 'location',
  'subType': 'AIRPORT',
  'name': 'AAF AIRPORT',
  'detailedName': 'HEIDELBERG/DE:AAF AIRPORT',
  'id': 'AHDB',
  'self': {'href': 'https://test.api.amadeus.com/v1/reference-data/locations/AHDB',
   'methods': ['GET']},
  'timeZoneOffset': '+01:00',
  'iataCode': 'HDB',
  'geoCode': {'latitude': 49.39362, 'longitude': 8.65556},
  'address': {'cityName': 'HEIDELBERG',
   'cityCode': 'HDB',
   'countryName': 'GERMANY',
   'countryCo

In [8]:
import mysql.connector
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [16]:
# Connect to MySQL

gc = mysql.connector.connect(
    host= '00.000.000.000',
    user= 'root',
    password= '00000',
)

In [17]:
# Return query as pandas dataframe

def sqlgc(_query):
    return pd.read_sql_query(_query, gc)

In [18]:
cursor = gc.cursor()

In [68]:
# Create a table for the airports we've collected -> identifying necessary primary key and data types

cursor.execute("""CREATE TABLE demo.eu_airports (
                id char(4) NOT NULL,
                airport_name varchar(150),
                iataCode char(3),
                city_name varchar(150),
                country_name varchar(70),
                country_code char(2),
                latitude float,
                longitude float,
                PRIMARY KEY (id)
                )""")

In [22]:
gc.commit()

In [27]:
# Insert data into recently created table

for airport in airports_list:
    
    id = airport['id']
    airport_name = airport['name']
    iataCode = airport['iataCode']
    city_name = airport['address']['cityName']
    country_name = airport['address']['countryName']
    country_code = airport['address']['countryCode']
    latitude = airport['geoCode']['latitude']
    longitude = airport['geoCode']['longitude']
   
    cursor.execute(f"""INSERT INTO demo.eu_airports (id, airport_name, iataCode, city_name, country_name, country_code, latitude, longitude) 
               VALUES ('{id}', "{airport_name}", '{iataCode}', "{city_name}", "{country_name}", '{country_code}', {latitude}, {longitude})
               """)
    gc.commit()

In [28]:
sqlgc("SELECT * FROM demo.eu_airports")

,id,airport_name,iataCode,city_name,country_name,country_code,latitude,longitude
0,AAAH,MERZBRUECK,AAH,AACHEN,GERMANY,DE,50.8233,6.18834
1,AACE,LANZAROTE,ACE,LANZAROTE,SPAIN,ES,28.9503,-13.60550
2,AAGB,AUGSBURG AIRPORT,AGB,MUNICH,GERMANY,DE,48.4253,10.93170
3,AAGE,WANGEROOGE,AGE,WANGEROOGE,GERMANY,DE,53.7833,7.91667
4,AAGF,LA GARENNE,AGF,AGEN,FRANCE,FR,44.1747,0.59056
...,...,...,...,...,...,...,...,...
252,AXRY,JEREZ AIRPORT,XRY,JEREZ,SPAIN,ES,36.7447,-6.06000
253,AZAO,LALBENQUE,ZAO,CAHORS,FRANCE,FR,44.3514,1.47528
254,AZAZ,ZARAGOZA,ZAZ,ZARAGOZA,SPAIN,ES,41.6661,-1.04166
255,AZQW,ZWEIBRUECKEN,ZQW,SAARBRUECKEN,GERMANY,DE,49.2094,7.40056


In [29]:
sqlgc("SELECT * FROM demo.eu_airports ORDER BY country_code")

,id,airport_name,iataCode,city_name,country_name,country_code,latitude,longitude
0,AAAH,MERZBRUECK,AAH,AACHEN,GERMANY,DE,50.8233,6.18834
1,AZQW,ZWEIBRUECKEN,ZQW,SAARBRUECKEN,GERMANY,DE,49.2094,7.40056
2,AAGB,AUGSBURG AIRPORT,AGB,MUNICH,GERMANY,DE,48.4253,10.93170
3,AAGE,WANGEROOGE,AGE,WANGEROOGE,GERMANY,DE,53.7833,7.91667
4,AXLW,LEMWERDER,XLW,LEMWERDER,GERMANY,DE,53.1300,8.61695
...,...,...,...,...,...,...,...,...
252,AFLR,PERETOLA,FLR,FLORENCE,ITALY,IT,43.8100,11.20500
253,ABZO,DOLOMITI,BZO,BOLZANO BOZEN,ITALY,IT,46.4603,11.32640
254,ACAG,ELMAS,CAG,CAGLIARI,ITALY,IT,39.2514,9.05417
255,ALCV,TASSIGNANO,LCV,LUCCA,ITALY,IT,43.8300,10.58000
